In [1]:
# Import required libraries
import pandas as pd
import time
import os
from autogluon.tabular import TabularPredictor

# Track start time
start_time = time.time()

# Load datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
extra_train_df = pd.read_csv("training_extra.csv")

# Combine train datasets
train_df = pd.concat([train_df, extra_train_df], ignore_index=True)

# Drop ID column from train set (AutoGluon handles it automatically)
test_ids = test_df["id"]
train_df.drop(columns=["id"], inplace=True)
test_df.drop(columns=["id"], inplace=True)

# Define target variable
target_col = "Price"

# Train AutoGluon Model
predictor = TabularPredictor(label=target_col, eval_metric="root_mean_squared_error").fit(
    train_data=train_df,
    time_limit=10 * 60 * 60,  # 10 hours
    presets="best_quality",  # Best quality models (Auto-stacking enabled)
    num_stack_levels=3,  # Increase stack levels for better performance
    num_bag_folds=5,  # Cross-validation
    ag_args_fit={"num_gpus": 1}  # Use GPU
)

predictor

No path specified. Models will be saved in: "AutogluonModels\ag-20250216_134236"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       13.35 GB / 31.82 GB (42.0%)
Disk Space Avail:   490.32 GB / 953.00 GB (51.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdo

KeyboardInterrupt: 

In [ ]:
# Get leaderboard (Top 10 models)
leaderboard = predictor.leaderboard(silent=True)
top_10_models = leaderboard.head(10)

# Print Top 10 Model Details
print("\n🔷 Top 10 Models from AutoGluon:")
print(top_10_models)

In [ ]:
# Generate predictions using the best model
preds = predictor.predict(test_df)

# Save submission file
submission = pd.DataFrame({"id": test_ids, "Price": preds})
submission.to_csv("submission.csv", index=False)

(_dystack pid=25540) 	Warning: Exception caused NeuralNetTorch_r89_BAG_L1 to fail during training... Skipping this model.
(_dystack pid=25540) 		Specified num_gpus per model base is more than the total: 0
(_dystack pid=25540) Detailed Traceback:
(_dystack pid=25540) Traceback (most recent call last):
(_dystack pid=25540)   File "C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1904, in _train_and_save
(_dystack pid=25540)     model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
(_dystack pid=25540)   File "C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1844, in _train_single
(_dystack pid=25540)     model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, total_resources=total_resources, **model_fit_kwargs)
(_dystack pid=25540)   File "C:\Users\Hi\anaconda3\envs\gpu_env\lib\site-packages\autogluon\core\models\abstract\abstract_